In [1]:
pip install pysentimiento

   ---------------------------------------- 0.0/608.4 kB ? eta -:--:--
   ---------------------------------------- 608.4/608.4 kB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/26.1 MB ? eta -:--:--
   -- ------------------------------------- 1.6/26.1 MB 7.7 MB/s eta 0:00:04
   ----- ---------------------------------- 3.4/26.1 MB 8.5 MB/s eta 0:00:03
   -------- ------------------------------- 5.5/26.1 MB 9.4 MB/s eta 0:00:03
   ------------ --------------------------- 8.1/26.1 MB 10.0 MB/s eta 0:00:02
   ----------------- ---------------------- 11.5/26.1 MB 11.2 MB/s eta 0:00:02
   ------------------------ --------------- 16.0/26.1 MB 12.9 MB/s eta 0:00:01
   -------------------------------- ------- 21.2/26.1 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------  26.0/26.1 MB 16.1 MB/s eta 0:00:01
   ---------------------------------------- 26.1/26.1 MB 15.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.9 MB ? eta -:--:--
   --

In [2]:
import pandas as pd
from pysentimiento import create_analyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Microsoft\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Microsoft\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv(r'Avaliacao_Produtos.csv')

In [4]:
analyzer = create_analyzer(task="sentiment", lang="pt")

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

C:\Users\Microsoft\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Microsoft\.cache\huggingface\hub\models--pysentimiento--bertweet-pt-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to reg

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [6]:
def get_sentiment(review):
    result = analyzer.predict(review)
   
    if result.output == "POS":
        return "bom"
    elif result.output == "NEG":
        return "ruim"
    else:
        return "neutro"

In [7]:
stop_words = set(stopwords.words('portuguese'))
stemmer = SnowballStemmer('portuguese')

In [8]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [stemmer.stem(token) for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

In [9]:
def get_top_words(review):
    tokens = preprocess_text(review)
    
    # Calculate sentiment for each word
    word_sentiments = []
    for word in set(tokens):
        # Analyze each word individually
        scores = analyzer.predict(word).probas
        compound = scores.get('POS', 0) - scores.get('NEG', 0)  # Simplified compound score
        if abs(compound) > 0.1:  # Ignore near-neutral words
            word_sentiments.append((word, compound))
    
    # Sort by sentiment score
    word_sentiments.sort(key=lambda x: x[1], reverse=True)
    
    # Get top 3 positive and negative words
    pos_words = [w for w, s in word_sentiments if s > 0][:3]
    neg_words = [w for w, s in word_sentiments if s < 0][:3]
    
    # Pad with None if fewer than 3 words
    pos_words += [None] * (3 - len(pos_words))
    neg_words += [None] * (3 - len(neg_words))
    
    return pos_words + neg_words

In [10]:
df['sentimento'] = df['review'].apply(get_sentiment)

In [11]:
word_columns = df['review'].apply(get_top_words)
word_columns = pd.DataFrame(word_columns.tolist(), 
                           columns=['palavra_pos1', 'palavra_pos2', 'palavra_pos3', 
                                    'palavra_neg1', 'palavra_neg2', 'palavra_neg3'], 
                           index=df.index)

In [12]:
df = pd.concat([df, word_columns], axis=1)
df.head()

,product_id,product_title,rating,review,upvotes,downvotes,sentimento,palavra_pos1,palavra_pos2,palavra_pos3,palavra_neg1,palavra_neg2,palavra_neg3
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,1-more flexible2-bass is very high3-sound clar...,1390,276,neutro,awesom,good,very,but,only,None
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super sound and good looking I like that prize,643,133,bom,good,and,looking,None,None,None
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Very much satisfied with the device at this pr...,1449,328,bom,awesom,wonderful,must,an,None,None
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,"Nice headphone, bass was very good and sound i...",160,28,bom,best,good,full,None,None,None
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Sound quality super battery backup super quali...,533,114,neutro,this,pric,and,None,None,None
